# UI4
updates
1. Better Sound Prediction

In [1]:
import os

current_path = os.getcwd()

# Mic device pick

In [2]:
import sounddevice as sd

def list_input_devices():
    devices = sd.query_devices()
    for i, device in enumerate(devices):
        if device['max_input_channels'] > 0:  # this is an input device
            print(f"Device #{i} name: {device['name']}")

# List available input devices (including microphones)
list_input_devices()
device_id = 2 # <<<  -  pick your mic

Device #1 name: 갤럭시 S2 마이크
Device #2 name: USB PnP Sound Device
Device #3 name: MacBook Air 마이크


# Mic Test
MIC TEST type 1 if you want to test

In [3]:
#MIC TEST type 1 if you want to test
test = 1
if test == 1:
    import numpy as np

    # Choose the device to use for recording
    duration = 2  # seconds

    # Create a buffer to store the audio data
    buffer = np.zeros((duration * 44100,))
    buffer_index = 0

    # Define a callback function to process the audio input
    def audio_callback(indata, frames, time, status):
        global buffer_index
        volume_norm = np.linalg.norm(indata) * 10
        print(f'\r{"|" * int(volume_norm)}', end='')  # print a simple "volume bar"

        # Store the incoming data in the buffer
        buffer[buffer_index:buffer_index+frames] = indata[:, 0]
        buffer_index += frames

    # Create a stream object
    stream = sd.InputStream(callback=audio_callback, device=device_id, channels=1, samplerate=44100)

    # Start the stream
    with stream:
        # Record for 3 seconds
        sd.sleep(duration * 1000)

    # Play back the recorded sound
    sd.play(buffer, samplerate=44100)

||||

## Pick the labels that you want to use here

## sound_rate (only raspberrypi use)

In [4]:
# text = '''pcm.!default {
#     type asym
#     capture.pcm "mic"
#     playback.pcm "speaker"
# }
# pcm.mic {
#     type plug
#     slave {
#         pcm "hw:2,0"
#     }
# }
# pcm.speaker {
#     type plug
#     slave {
#         pcm "hw:1,0"
#     }
# }'''

# # 파일 경로와 이름 설정
# file_path = "/home/pi/.asoundrc"

# # 텍스트 파일 생성 및 저장
# with open(file_path, "w") as file:
#     file.write(text)

# print("Text saved to:", file_path)

# Model Path

In [5]:
# Relative path to the model file
path_to_model = '../@AI/AI/models/Batch4-1k_004.pth'
print("Path to the model:", path_to_model)

Path to the model: ../@AI/AI/models/Batch4-1k_004.pth


In [6]:
import torch
import torch.nn as nn
from torchvision.models import resnet18

#One GPU or CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(512, 14)
try:
    state_dict = torch.load(path_to_model, map_location=device)
    new_state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
    model.load_state_dict(new_state_dict)
    model = model.to(device)
    model = model.eval()
    print("Model successfully loaded deivce : ",device)
except:
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    print("Failed to load the model. Please check the model file.")

/Users/owo/anaconda3/envs/torchenv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/owo/anaconda3/envs/torchenv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Model successfully loaded deivce :  cpu


# Data processing func

In [7]:
import torchaudio
from torchvision import transforms  # Import the transforms module

SAMPLE_RATE = 22050

# Normalize
class MinMaxNormalize(nn.Module):
    def __init__(self, min_val=None, max_val=None):
        super(MinMaxNormalize, self).__init__()
        self.min_val = min_val
        self.max_val = max_val

    def forward(self, tensor):
        if self.min_val is None or self.max_val is None:
            min_val = torch.min(tensor)
            max_val = torch.max(tensor)
        else:
            min_val = self.min_val
            max_val = self.max_val
        
        normalized_tensor = (tensor - min_val) / (max_val - min_val)
        return normalized_tensor


# Mono To Color (for 3 channels)
class MonoToColor(nn.Module):
    def __init__(self, num_channels=3):
        super(MonoToColor, self).__init__()
        self.num_channels = num_channels

    def forward(self, tensor):
        return tensor.repeat(self.num_channels, 1, 1)

# Transform + appy all func
transformation = transforms.Compose([
    torchaudio.transforms.MelSpectrogram(sample_rate=SAMPLE_RATE, n_mels=40),# higher the better but more complex. For talking we use 128, for sound effect, about 40.
    torchaudio.transforms.AmplitudeToDB(stype='power', top_db=80),
    MinMaxNormalize(),
    MonoToColor()
])

# Speech to text

In [8]:
import pyaudio
import speech_recognition as sr
import time

def list_input_devices():
    audio = pyaudio.PyAudio()
    for i in range(audio.get_device_count()):
        device_info = audio.get_device_info_by_index(i)
        if device_info['maxInputChannels'] > 0:  # This is an input device
            print(f"Device #{i} name: {device_info['name']}")

def record_audio():
    audio = pyaudio.PyAudio()
    stream = audio.open(format=pyaudio.paInt16, channels=1, rate=16000, frames_per_buffer=1024, input=True , input_device_index=device_id)
    frames = []
    for i in range(0, int(16000 / 1024 * 3)):
        data = stream.read(1024)
        frames.append(data)
    stream.close()
    audio.terminate()  # close the PyAudio object
    return b''.join(frames)
def transcribe_audio(audio):
    recognizer = sr.Recognizer()
    audio_data = sr.AudioData(audio, 16000, 2)
    try:
        text = recognizer.recognize_sphinx(audio_data)
        
        return text
    except:
        return None
def main(times):
    for i in range(0, times):
        audio = record_audio()
        text = transcribe_audio(audio)
        print(text)
TIMES = 0
if __name__ == "__main__":
    main(TIMES)

list_input_devices()

Device #1 name: 갤럭시 S2 마이크
Device #2 name: USB PnP Sound Device
Device #3 name: MacBook Air 마이크


# SOUND EFFECT

In [9]:
import time
import torch.nn.functional as F
import sounddevice as sd
global count
count = 0
print("device : ",device)
## print every labels
def continuous_sound_prediction(model, device, transformation, sample_rate):

    global count

    count = count + 1

    labels = [
        "nothing a", "Car Horn", "Bell Ring", "Dog Bark", "nothing b", 
        "nothing c", "Glass Shatter", "Nothing d", "Nothing e", "Door Nock", 
        "Nothing f", "Nothing g", "Siren", "Nothing h"
    ]

    duration = 2.0  # seconds
    recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, device=device_id)
    sd.wait()
    
    # Preprocessing
    recording = torch.from_numpy(recording).float().transpose(0, 1)
    if recording.shape[0] > 1:
        recording = torch.mean(recording, dim=0, keepdim=True)
    recording = nn.functional.pad(recording, (0, sample_rate - recording.shape[1]))
    
    # Transformation
    recording = transformation(recording)
    
    # Prediction
    model.eval()
    with torch.no_grad():
        recording = recording.to(device)
        outputs = model(recording.unsqueeze(0))
        #probabilities = F.softmax(outputs, dim=1)
        probabilities = torch.sigmoid(outputs)

        _, predicted = outputs.max(1)
    # Print results
    probs = [f"{label} {prob:.2%}" for label, prob in zip(labels, probabilities[0])]
    print(f"{count} / {' / '.join(probs)}")

    max_prob, predicted_label_idx = probabilities[0].max(0)
    max_prob_label = labels[predicted_label_idx]

    if max_prob > 0.0:
        return max_prob_label, max_prob, probabilities
    else:
        return "NONE" , 0 , ""


device :  cpu


In [10]:
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtCore import QThread, pyqtSignal
from PyQt5.QtGui import QMovie

from time import sleep
import torch
import torch.nn as nn
import pyaudio
import torchaudio
import sounddevice as sd
import torchvision.transforms as transforms
from torchvision.models import resnet18
from torch.autograd import Variable


###### INIT ######
p = pyaudio.PyAudio()
sample_rate = SAMPLE_RATE
target_sample_rate = SAMPLE_RATE
mode = 0 ## 1 starts with sound effect, 0 starts with stt
print(" 결과 값 : ")


# Sound Analysis class running on a separate thread
class SoundAnalysis(QThread):
    # Define a pyqtSignal with str type, which will be used to send the analysis results to the main thread
    result_signal = pyqtSignal(str, float)  # Add a float type for the probability

    def __init__(self, model, device, transformation, sample_rate):
        QThread.__init__(self)
        self.model = model
        self.device = device
        self.transformation = transformation
        self.sample_rate = sample_rate
        

    def run(self):## Main code
        count = 0
        text_str = ""
        while True:

            if mode == True:
                text_str = "" #empty the text strings

                predicted_label, predicted_confidence, probabilities = continuous_sound_prediction(model, device, transformation, SAMPLE_RATE) 
                if mode == True:
                    try:               
                        self.result_signal.emit(predicted_label, predicted_confidence)
                        #vibration()
                    except:
                        self.result_signal.emit(predicted_label, predicted_confidence)
                    count = count + 1
            if mode == False:

                times = 2
                for _ in range(0, times):
                    audio = record_audio()
                    text = transcribe_audio(audio)
                    print(text)
                    text_str += text + " "
                    if mode == True: ## This is checking again if the button is pushed. If I dont add this, it will reset_label and show the text_str again before going to mode True
                        break
                    self.result_signal.emit(text_str,0)
        
    def reset_label(self):# Restting the labels when the button is pressed
        self.result_signal.emit("켜는중", 0)
        
class Ui_MainWindow(object):
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(640, 480)
        self.received_text = ""
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        
        # Create a QScrollArea
        self.scrollArea = QtWidgets.QScrollArea(self.centralwidget)
        self.scrollArea.setGeometry(QtCore.QRect(10, 300, 611, 121))
        self.scrollArea.setWidgetResizable(True)
        self.scrollArea.setObjectName("scrollArea")

        # Create the label inside the QScrollArea
        self.label = QtWidgets.QLabel(self.scrollArea)
        self.label.setAlignment(QtCore.Qt.AlignCenter)
        self.label.setObjectName("text_label")
        self.label.setWordWrap(True)
        self.scrollArea.setWidget(self.label)

        self.label_2 = QtWidgets.QLabel(self.centralwidget)
        self.label_2.setGeometry(QtCore.QRect(10, 10, 441, 261))
        self.label_2.setText("")
        self.label_2.setObjectName("imgae_image")

        self.pushButton = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton.setGeometry(QtCore.QRect(460, 10, 171, 111))
        self.pushButton.setObjectName("pushButton")
        MainWindow.setCentralWidget(self.centralwidget)

        #english_text_label
        self.new_label = QtWidgets.QLabel(self.centralwidget)
        self.new_label.setGeometry(QtCore.QRect(40, 200, 561, 201))
        self.new_label.setText("")
        self.new_label.setAlignment(QtCore.Qt.AlignCenter)  # 텍스트 중앙 정렬
        self.new_label.setObjectName("english_text_label")
        self.new_label.hide()  # Hide the new label initially
        #english_image_label
        self.new_label_2 = QtWidgets.QLabel(self.centralwidget)
        self.new_label_2.setGeometry(QtCore.QRect(50, 40, 144, 130))
        self.new_label_2.setPixmap(QtGui.QPixmap("english_image.png"))
        self.new_label_2.setAlignment(QtCore.Qt.AlignCenter)
        self.new_label_2.setObjectName("english_image_label")
        

        MainWindow.setCentralWidget(self.centralwidget)
        self.menubar = QtWidgets.QMenuBar(MainWindow)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 800, 29))
        self.menubar.setObjectName("menubar")
        MainWindow.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(MainWindow)
        self.statusbar.setObjectName("statusbar")
        MainWindow.setStatusBar(self.statusbar)

        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)

        self.pushButton.clicked.connect(self.onPushButtonClicked)

        # Initialize SoundAnalysis and connect the result_signal with the updateLabel function
        self.sound_analysis = SoundAnalysis(model, device, transformation, SAMPLE_RATE)
        self.sound_analysis.result_signal.connect(self.updateLabel)
        self.sound_analysis.result_signal.connect(self.updateLabel2)
        self.sound_analysis.start()  # Start the sound analysis thread
        
        self.is_new_labels_visible = False

        

    def onPushButtonClicked(self): ######BUTTON#######
        global mode
        mode = not mode
        self.new_label_2.hide()  # Hide the new label initially
        self.sound_analysis.reset_label()#reset the screen (its connected to the sound analysis class)
        print("mode now :  ",mode)

    def updateLabel(self, predicted_label, predicted_confidence):
        #print("Received signal")  # Print message when signal is received
        if predicted_confidence == 0:
            self.label.setText(f"{predicted_label}")
        else:
            self.label.setText(f"{predicted_label}  {predicted_confidence*100:.2f}%")
        #print(predicted_label)
        
        # After updating the text, ensure the QScrollArea scrolls to the bottom
        self.scrollArea.verticalScrollBar().setValue(self.scrollArea.verticalScrollBar().maximum())

        
        
    def updateLabel2(self, predicted_label):
        relative_image_folder_path = "../image_file/GIF" #\ only works for windows
        image_folder_path= os.path.join(current_path, relative_image_folder_path)
        full_file_name = os.path.join(image_folder_path, f"{predicted_label}.gif")
        self.movie = QMovie(full_file_name)
        self.label_2.setMovie(self.movie)
        self.movie.start()

        #print(predicted_label)


    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "MainWindow"))
        self.label.setFont(QtGui.QFont("AppleSystemUIFont",20))
        self.label.setStyleSheet("Color : black")
        self.pushButton.setText(_translate("MainWindow", "PushButton"))
        
try:
    import RPi.GPIO as GPIO
    print("RPi.GPIO loaded successfully")
    import threading
    print("threading loaded successfully")

    class VibrationController:
        def __init__(self):
            # 핀 번호 설정
            self.red_button_pin = 17
            self.yellow_button_pin = 22
            self.green_button_pin = 27
            self.vibration_motor_pin = 18
            
            # 진동 세기 초기화
            self.vibration_intensity_temp = 100
            self.vibration_intensity = 0
            
            # 이전 스위치 상태 초기화
            self.prev_red_button_state = GPIO.HIGH
            self.prev_yellow_button_state = GPIO.HIGH
            self.prev_green_button_state = GPIO.HIGH
            
            self.debounce_time = 0.2
            
            # 진동 상태를 저장하는 변수
            self.vibration_on = False
            
            # GPIO 초기화
            GPIO.setmode(GPIO.BCM)
            GPIO.setup(self.red_button_pin, GPIO.IN, pull_up_down=GPIO.PUD_UP)
            GPIO.setup(self.yellow_button_pin, GPIO.IN, pull_up_down=GPIO.PUD_UP)
            GPIO.setup(self.green_button_pin, GPIO.IN, pull_up_down=GPIO.PUD_UP)
            GPIO.setup(self.vibration_motor_pin, GPIO.OUT)
            
            # PWM 설정
            self.pwm_frequency = 1000
            self.pwm = GPIO.PWM(self.vibration_motor_pin, self.pwm_frequency)
            self.pwm.start(self.vibration_intensity)
        
        def display_vibration_intensity(self):
            print(f"진동 세기: {self.vibration_intensity}")
        
        def adjust_vibration_intensity(self, button_pin):
            if button_pin == self.red_button_pin:
                self.vibration_intensity = self.vibration_intensity_temp
            elif button_pin == self.yellow_button_pin:
                self.vibration_intensity = max(self.vibration_intensity - 10, 0)
            elif button_pin == self.green_button_pin:
                self.vibration_intensity = min(self.vibration_intensity + 10, 100)
            
            self.pwm.ChangeDutyCycle(self.vibration_intensity)
            self.display_vibration_intensity()
        
        def run(self):
            try:
                while True:
                    red_button_state = GPIO.input(self.red_button_pin)
                    yellow_button_state = GPIO.input(self.yellow_button_pin)
                    green_button_state = GPIO.input(self.green_button_pin)
            
                    if red_button_state != self.prev_red_button_state:
                        time.sleep(self.debounce_time)
                        if red_button_state != GPIO.input(self.red_button_pin):
                            
                            if not self.vibration_on:
                                self.vibration_on = True
                                self.adjust_vibration_intensity(self.red_button_pin)
                            else:
                                self.vibration_intensity_temp = self.vibration_intensity
                                self.vibration_intensity = 0
                                self.pwm.ChangeDutyCycle(self.vibration_intensity)
                                self.display_vibration_intensity()
                                self.vibration_on = False
            
                        self.prev_red_button_state = red_button_state
            
                    if yellow_button_state != self.prev_yellow_button_state:
                        time.sleep(self.debounce_time)
                        if yellow_button_state != GPIO.input(self.yellow_button_pin):
                            self.adjust_vibration_intensity(self.yellow_button_pin)
                        self.prev_yellow_button_state = yellow_button_state
            
                    if green_button_state != self.prev_green_button_state:
                        time.sleep(self.debounce_time)
                        if green_button_state != GPIO.input(self.green_button_pin):
                            self.adjust_vibration_intensity(self.green_button_pin)
                        self.prev_green_button_state = green_button_state
            
                    time.sleep(0.01)  
            
            except KeyboardInterrupt:
                self.pwm.stop()
                GPIO.cleanup()
except:
    print("RPi.GPIO import failed. No vibration")

if __name__ == "__main__":
    import sys
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    try:
        vibration_controller = VibrationController()
        vibration_thread = threading.Thread(target=vibration_controller.run)
        vibration_thread.start()
    except:
        print("pass vibration")
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    MainWindow.show()
    sys.exit(app.exec_())

 결과 값 : 
RPi.GPIO import failed. No vibration
pass vibration


qt.qpa.fonts: Populating font family aliases took 93 ms. Replace uses of missing font family "AppleSystemUIFont" with one that exists to avoid this cost. 
2023-09-05 09:17:38.781 python[30580:6962705] IMKClient Stall detected, *please Report* your user scenario attaching a spindump (or sysdiagnose) that captures the problem - (imkxpc_bundleIdentifierWithReply:) block performed very slowly (1.32 secs).


what you do you love you
mode now :   True
so
1 / nothing a 17.94% / Car Horn 0.13% / Bell Ring 2.15% / Dog Bark 48.78% / nothing b 1.30% / nothing c 1.11% / Glass Shatter 0.26% / Nothing d 1.91% / Nothing e 5.46% / Door Nock 2.64% / Nothing f 0.01% / Nothing g 0.70% / Siren 0.01% / Nothing h 4.27%
2 / nothing a 22.98% / Car Horn 0.01% / Bell Ring 0.36% / Dog Bark 4.09% / nothing b 2.21% / nothing c 36.72% / Glass Shatter 0.07% / Nothing d 0.09% / Nothing e 22.39% / Door Nock 0.03% / Nothing f 0.00% / Nothing g 1.06% / Siren 0.29% / Nothing h 4.47%
3 / nothing a 89.99% / Car Horn 0.01% / Bell Ring 2.09% / Dog Bark 1.48% / nothing b 2.96% / nothing c 68.94% / Glass Shatter 0.05% / Nothing d 0.08% / Nothing e 12.04% / Door Nock 0.04% / Nothing f 0.00% / Nothing g 11.40% / Siren 0.00% / Nothing h 28.47%
4 / nothing a 78.09% / Car Horn 0.01% / Bell Ring 0.66% / Dog Bark 6.11% / nothing b 2.25% / nothing c 26.66% / Glass Shatter 0.07% / Nothing d 0.06% / Nothing e 33.23% / Door Nock 0.14% /

SystemExit: 0

/Users/owo/anaconda3/envs/torchenv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


436 / nothing a 3.57% / Car Horn 0.11% / Bell Ring 1.10% / Dog Bark 39.55% / nothing b 0.36% / nothing c 0.35% / Glass Shatter 0.09% / Nothing d 0.94% / Nothing e 6.07% / Door Nock 0.47% / Nothing f 0.01% / Nothing g 0.95% / Siren 0.02% / Nothing h 10.42%


# 가능한 폰트 확인

In [ ]:
from PyQt5.QtGui import QFontDatabase

print(QFontDatabase().families())